In [2]:
import pandas as pd

In [3]:
import os
os.getcwd()

'C:\\Users\\Work\\Desktop\\github\\CodeLou\\PythonDataAnalysis'

In [13]:
df = pd.read_excel(r"C:\Users\Work\Downloads\FSIS-Recall-Summary-2014.xlsx")
print (df)

        CY 2014 Recalls     Unnamed: 1    Unnamed: 2  \
0           Recall Date  Recall Number  Recall Class   
1   2014-01-10 00:00:00       001-2014             I   
2   2014-01-13 00:00:00       002-2014             I   
3   2014-01-15 00:00:00       003-2014             I   
4   2014-01-17 00:00:00       004-2014            II   
..                  ...            ...           ...   
90  2014-12-27 00:00:00       090-2014             I   
91  2014-12-27 00:00:00       091-2014             I   
92  2014-12-29 00:00:00       092-2014             I   
93  2014-12-31 00:00:00       093-2014             I   
94  2014-12-31 00:00:00       094-2014             I   

                                 Unnamed: 3           Unnamed: 4  \
0                                   Product    Reason for Recall   
1   Mechanically Separated Chicken Products           Salmonella   
2                     Various Beef Products                Other   
3                               Beef Franks  Undeclared